# OpenStreetMap Wrangling with MongoDB

### Justin Rogers

Map Area: Boulder CO, United States  
[Boulder on OpenStreetMaps](http://www.openstreetmap.org/relation/112298)  
[Overpass API Link for Boulder](http://overpass-api.de/api/map?bbox=-105.4142,39.9490,-105.0653,40.1099)

1. [Problems Encountered in the Map](#section1)
2. [Data Overview](#section2)
3. [Additional Ideas](#section2)

##  <a id='section1'>1. Problems Encountered in the Map</a>

## <a id='section2'>2. Data Overview</a>

In [16]:
import os
print "boulder.osm file is %.1fMB" % (float(os.path.getsize('boulder.osm'))/1000000)

boulder.osm file is 91.5MB


## <a id='section3'>3. Additional Ideas</a>